DSPy reduces time spent prompting by providing you with boilerplate prompting that frames your tasks, which you define with "Signatures". __Signatures__ are a way of expressing what you want an LLM to do by defining the desired input and outputs.  

For examples:

`question -> answer` 

`sentence -> sentiment: bool`

`document -> summary`

`novella -> tldr`

`image -> caption`

`image -> label`


Signatures can also be defined as a class. 